# CNN to dog breeds

In this notebook, I aim to apply Tensorflow and Pytorch AI frameworks to classify different Dog breed given an image.

## Data

In [ ]:
# Import libraries
import pandas as pd
import glob

In [36]:
# Load the dog images (.jpg extension)
dog_images = glob.glob("dogimages/**/*.jpg", recursive=True)
num_dog_images = len(dog_images)
print(f"There are {num_dog_images} dog images.")

There are 8351 dog images.


In [ ]:
# Train
train_dog_images = glob.glob("dogimages/train/**/*.jpg", recursive=True)
num_train_dog_images = len(train_dog_images)
print(f"There are {num_train_dog_images} dog images for train.")

There are 6680 dog images for train.


In [ ]:
# Validation
val_dog_images = glob.glob("dogimages/valid/**/*.jpg", recursive=True)
num_val_dog_images = len(val_dog_images)
print(f"There are {num_val_dog_images} dog images for validation.")

There are 835 dog images for validation.


In [ ]:
# Test
test_dog_images = glob.glob("dogimages/test/**/*.jpg", recursive=True)
num_test_dog_images = len(test_dog_images)
print(f"There are {num_test_dog_images} dog images for test.")

There are 836 dog images for test.


In [42]:
# We check that there is no any wrong or missing image
assert num_dog_images == num_train_dog_images + num_val_dog_images + num_test_dog_images

In [22]:
# Different breeds
dog_breeds = glob.glob("dogimages/train/*")
dog_breeds = [dog_breed_tmp.split(".")[1] for dog_breed_tmp in dog_breeds]

**TODO**: Show some image

## CNN

In [43]:
# Import libraries
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np

### Tensorflow

In [44]:
# Import libraries
import tensorflow as tf

**Step 1**: Load data

In [46]:
# Tamaño al que se redimensionarán todas las imágenes
IMG_SIZE = (128, 128)
# Número de imágenes por batch (más grande = más RAM)
BATCH_SIZE = 32
# Carga las imágenes desde la carpeta "data/train"
train_ds = tf.keras.utils.image_dataset_from_directory(
    "dogimages/train",           # Ruta a la carpeta con subcarpetas por clase
    image_size=IMG_SIZE,    # Redimensiona todas las imágenes a 128x128
    batch_size=BATCH_SIZE,  # Agrupa en lotes de 32 imágenes
    shuffle=True            # Mezcla los datos para entrenar mejor
)
# Lo mismo para el conjunto de validación
val_ds = tf.keras.utils.image_dataset_from_directory(
    "dogimages/valid",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)
# Y para test (final, sin shuffle para evaluación)
test_ds = tf.keras.utils.image_dataset_from_directory(
    "dogimages/test",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)
# Obtiene automáticamente los nombres de las clases (carpetas)
class_names = train_ds.class_names
print("Clases encontradas:", class_names)

Found 6680 files belonging to 133 classes.
Found 835 files belonging to 133 classes.
Found 836 files belonging to 133 classes.
Clases encontradas: ['001.Affenpinscher', '002.Afghan_hound', '003.Airedale_terrier', '004.Akita', '005.Alaskan_malamute', '006.American_eskimo_dog', '007.American_foxhound', '008.American_staffordshire_terrier', '009.American_water_spaniel', '010.Anatolian_shepherd_dog', '011.Australian_cattle_dog', '012.Australian_shepherd', '013.Australian_terrier', '014.Basenji', '015.Basset_hound', '016.Beagle', '017.Bearded_collie', '018.Beauceron', '019.Bedlington_terrier', '020.Belgian_malinois', '021.Belgian_sheepdog', '022.Belgian_tervuren', '023.Bernese_mountain_dog', '024.Bichon_frise', '025.Black_and_tan_coonhound', '026.Black_russian_terrier', '027.Bloodhound', '028.Bluetick_coonhound', '029.Border_collie', '030.Border_terrier', '031.Borzoi', '032.Boston_terrier', '033.Bouvier_des_flandres', '034.Boxer', '035.Boykin_spaniel', '036.Briard', '037.Brittany', '038.Bru

**Step 2**: Normalizar imágenes y preparar el pipeline de datos

In [48]:
# TensorFlow puede optimizar automáticamente el número de hilos al cargar datos
AUTOTUNE = tf.data.AUTOTUNE

# 🔧 Función que convierte los píxeles de int [0,255] a float [0.0,1.0]
def preprocess(ds):
    return ds.map(
        lambda x, y: (tf.cast(x, tf.float32) / 255.0, y),  # normaliza imágenes
        num_parallel_calls=AUTOTUNE                        # usa paralelismo para acelerar
    )

# Aplicamos procesamiento al dataset de entrenamiento
train_ds = (
    preprocess(train_ds)            # normaliza imágenes
    .cache()                        # guarda los datos en memoria tras primera pasada
    .shuffle(1000)                  # mezcla aleatoriamente (importante para entrenamiento)
    .prefetch(buffer_size=AUTOTUNE)  # prepara el siguiente batch mientras se entrena este
)

# Validación (sin shuffle, pero también normaliza y acelera carga)
val_ds = (
    preprocess(val_ds)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

# Test (igual que validación)
test_ds = (
    preprocess(test_ds)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

In [ ]:
# Creamos una red secuencial: capa tras capa
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(*IMG_SIZE, 3)),  # Imagen de entrada: (128, 128, 3)

    # Primer bloque conv: 32 filtros, tamaño 3x3, ReLU
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),  # Reduce tamaño (64x64)

    # Segundo bloque conv
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),  # Reduce tamaño (32x32)

    # Tercer bloque conv
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),  # Reduce tamaño (16x16)

    # Aplanamos la imagen 3D a un vector 1D
    tf.keras.layers.Flatten(),

    # Capa densa con 128 neuronas
    tf.keras.layers.Dense(128, activation='relu'),

    # Regularización: apagamos 50% de neuronas para evitar overfitting
    tf.keras.layers.Dropout(0.5),

    # Capa de salida: tantas neuronas como clases, activación softmax (probabilidades)
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

# Compilamos el modelo: definimos cómo se entrena
model.compile(
    optimizer='adam',                            # Optimizador adaptativo
    loss='sparse_categorical_crossentropy',      # Pérdida para etiquetas como enteros
    metrics=['accuracy']                         # Métrica a mostrar durante el entrenamiento
)

# Muestra resumen de capas, parámetros, formas, etc.
model.summary()

### Pytorch